In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reconstruction import Model, TopKLayer
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import synthesizer
import os
import itertools
from torch import optim

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [4]:
def getImage(img_path, p, epochs=100):
    m = Model(img_path, topk=p['topk'], device=p['device'], dimensions=p['dimensions'], mode=p['mode'])
    I = torch.rand((1, 3, m.dimensions[0], m.dimensions[1])).to(device)
    I = I.requires_grad_(True)
    optimizer = optim.LBFGS([I], lr=1)

    r = 0
    while r < epochs:
        def closure():
            optimizer.zero_grad()
            I.grad = None
            m(I)
            loss = m.loss()
            loss.backward(retain_graph=True)
            #print(f'I gradient: {I.grad}')
            return loss
        optimizer.step(closure)

        r += 1
        print(f'\rEpoch {r}: Loss {m.loss().item()}', end='')
    print()
    return I

In [5]:
def get_path(img_name):
    img_ext = '.jpg'
    img_path = os.path.join('./', img_name + img_ext)
    return img_path, img_name, img_ext

In [6]:
parameters = {
    'topk': 0.05,
    'device': device,
    'dimensions': (500, 500), 
    'mode': 'topk'
}

In [7]:
topks = [0.05, 0.5, 0.95]
modes = ['topk', 'non-topk', 'both']
img_names = ['jeep1', 'rocks']

for p in itertools.product(topks, modes, img_names):
    print(p)
    topk = p[0]
    mode = p[1]
    img_path, img_name, img_ext = get_path(p[2])
    
    parameters['topk'] = topk
    parameters['mode'] = mode

    I = getImage(img_path, parameters, epochs=100)
    transforms.ToPILImage()(np.clip(I.clone().detach().cpu().squeeze(0).numpy().transpose(1, 2, 0), 0, 1)).save(os.path.join('./results', f'{img_name}-{topk}-{mode}{img_ext}'))

(0.5, 'topk', 'jeep1')


/home/mrontio/uni/differentiable/labs/venv/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch 100: Loss 84011.8046875
(0.5, 'topk', 'rocks')
Epoch 100: Loss 85615.4062555
(0.5, 'non-topk', 'jeep1')
Epoch 100: Loss 831537.0625
(0.5, 'non-topk', 'rocks')
Epoch 100: Loss 905298.9375
(0.5, 'both', 'jeep1')
Epoch 100: Loss 113.34915924072266
(0.5, 'both', 'rocks')
Epoch 100: Loss 0.6456871628761292
(0.95, 'topk', 'jeep1')
Epoch 100: Loss 804.5863647460938
(0.95, 'topk', 'rocks')
Epoch 100: Loss 4727.38330078125
(0.95, 'non-topk', 'jeep1')
Epoch 100: Loss 78486.515625
(0.95, 'non-topk', 'rocks')
Epoch 100: Loss 82117.890625
(0.95, 'both', 'jeep1')
Epoch 100: Loss 241.20191955566406
(0.95, 'both', 'rocks')
Epoch 100: Loss 0.6387669444084167
